# Animationen in matplotlib

Die Bewegungsgleichung eines gedämpften harmonischen Oszillators 

$$
\ddot{x}(t) + 2\gamma\omega_0 \dot{x}(t) + \omega_0^2 x(t) = 0
$$

führt für $\gamma = \frac{1}{10}$ und $\omega_0=1$ zu der Lösung:

$$
x(t) = \left(\frac{\sqrt{11} \sin{\left(\frac{3 \sqrt{11} t}{10} \right)}}{33} + \cos{\left(\frac{3 \sqrt{11} t}{10} \right)}\right) e^{- \frac{ t}{10}}
$$

Wir wollen die Bewegung als Animation darstellen:

<center><img src="figuren/pendel.png" width="400" align="center"><img src="figuren/pendel.gif" width="400" align="center"></center>

In [ ]:
%matplotlib inline

# Benötigte Module
import numpy as np
import matplotlib.pyplot as plt

# Für Animationen notwendig:
import matplotlib.animation as ma

In [ ]:
# Position der Pendelbewegung und einfacher Plot:
def x(t):
    return (np.sqrt(11) * np.sin(3 * np.sqrt(11) * t / 10) / 33 + 
            np.cos(3 * np.sqrt(11) * t / 10)) * np.exp(-t / 10)

# statischer Plot der Bewegung für die ersten 20s:
t_n = np.linspace(0.0, 20.0, 100)
x_n = x(t_n)

fig, ax = plt.subplots(figsize=(4, 2), tight_layout=True)
ax.set_xlim(0, 20)
ax.set_ylim(-1, 1)
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$x(t)$')
ax.grid()

ax.plot(t_n, x_n, lw=2, color='blue')

## Grundlegende Ideen für animierte Plots

Eine Animation wird erzeugt, indem einzelne Bilder nacheinander dargestellt werden.

In [ ]:
# Bereitstellung der Daten; oft als Arrays
t_n = np.linspace(0.0, 20.0, 100)
x_n = x(t_n)

# Grundgerüst der Animation:
fig, ax = plt.subplots(figsize=(4, 2), tight_layout=True)
ax.set_xlim(0, 20)
ax.set_ylim(-1, 1)

# Die Animation ist die Hintereinanderdarstellung einzelner Bilder.
# Hier werden die Bilder durch Arrayelemente eines Liniendiagramms
# erzeugt:
frame = 55
ax.plot(t_n[:frame], x_n[:frame], lw=2, color='blue')
#ax.plot(t_n[frame], x_n[frame], 'o', markersize=10, color='red')

## Vom statischen Plot zur Animation in 6 Schritten

Zur Umsetzung obiger Animationsidee stellt `matplotlib` die Funktion `ma.FuncAnimation` zur Verfügung.

In [ ]:
# 1. Aufbereitung der Daten
nseconds = 20
nframes = 400
t_n = np.linspace(0.0, nseconds, nframes)
x_n = x(t_n)

# 2. Initialisierung des Funktionsgerüstes:
# Für Animationen benötigen wir Figuren und Achsen:
fig, ax = plt.subplots(figsize=(4, 2), tight_layout=True, dpi=200)

ax.set_xlim(0., 20.0)
ax.set_ylim(-1., 1.)
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$x(t)$')
ax.grid()

# 3. Deinition zu animierender Elemente
# Hier erzeugen Sie 'leere' Elemente mit evtl. Parametern (Strichdicke etc.)
line, = ax.plot([], [], lw=2, color='blue') # evtl. mit Stilelementen
dot, = ax.plot([], [], 'o', markersize=10, color='red') # evtl. mit Stilelementen

# Gegenüber dem Video wird hier die Positionsangabe des Textes unabhängig von
# den Werten in 'xlim' und 'ylim' angegeben, also unabhängig vom Koordinatensystem
# der Daten. Die Positionsangabe erfolgt stattdessen im Koordinatensystem der Figur.
# Das ist immer dann sinnvoll, wenn man von konkreten Datenwerten unabhängig sein
# möchte.
# Bitte im Netz hierzu u.a. die Seite 
# https://matplotlib.org/stable/users/explain/artists/transforms_tutorial.html
# befragen.
#text = ax.text(0.4, 0.85, "", 
#               bbox={'facecolor':'white','edgecolor':'black','pad':3},
#               ha='left', 
#               transform=ax.transAxes)

text = ax.text(7, 0.7, "", 
               bbox={'facecolor':'white'},
               )

# 4. definiere Zeitintervall zwischen den Bildern
#    der Animation; meist festgelegt durch die
#    gewünsche Gesamtdauer einer Animation
fps = nframes / nseconds       # frames (Bilder) pro Sekunde
interval_frames = 1000 / fps  # Zeitinterval zwischen
                              # zwei Bildern in Millisekunden

# 4. Definition der Funktion für die einzelnen Bilder der Animation
def drawframe(frame):
    line.set_data(t_n[:frame], x_n[:frame])
    dot.set_data([t_n[frame]], [x_n[frame]])
    text.set_text(f't = {frame / fps:05.2f} s')

    # immer eine 'Liste' aller modifizierten Elemente zurückgeben!
    return [line, dot, text]

# 5. Aufruf von FuncAnimation zur Erzeugung der Animation
animation = ma.FuncAnimation(fig=fig, 
                             func=drawframe, 
                             frames=len(t_n),
                             interval=interval_frames,
                             blit=True)

# Das 'blit=True' sollte bei Animationen, die man sich
# direkt am Bildschirm (oder im Notebook) ansehen möchte,
# gesetzt werden. Es sorgt dafür dass in der Darstellung
# nur Elemente , die sich zwischen frames verändern neu gezeichnet
# werden. Wenn man es nicht setzt, bekommt man u.U. ruckelige
# Darstellungen. Bei Animationen, die man nur abspeichert und später
# anschaut, ist es nicht wichtig. In diesem Fall werden die
# Programme in komplizierteren Föllen wie hier u.U. einfacher.

# 6. Anzeigen der Animation (optional)

# Das folgende (bis plt.close) ist im Notebook notwendig:
from IPython.display import HTML
html = HTML(animation.to_jshtml())

display(html)
plt.close()

# Für die Darstellung des Plots in einem Skript sollte
# ein einfaches 'plt.plot()' ausreichen. Wenn es nicht klappt
# (z.B. innerhalb einer bestimmten IDE), am einfachsten Dr. Google
# befragen.

# 7. Abspeichern der Animation als Video auf der Platte (optional)
# Das ABspeichern als bewegtes gif sollte bei jedem verfügbar sein:
#animation.save('pendel.gif', writer='pillow', fps=fps)

# Zum Abspeichern als mp4-Video (oder anderer Videoformate) benötigt
# man externe Programme wie z.B. ffmepg. Hierzu evtl bei Google
# nachschauen

# animation.save('ani_neu.mp4', writer='ffmpeg', fps=fps)

## Anhänge / Anmerkungen

## A.1 Animation anderer Plottypen als Liniendiagramme

Ich habe Ihnen primär gezeigt, wie Sie Plots, die in `matplotlib` mit dem `plot`-Kommando erstellt werden, animiert werden können. Dies sind in erster Linie zweidimensionale Linendiagramme. Daneben existieren noch zahlreiche weitere Arten wissenschaftlicher Plots, z.B. Scatter-Plots oder Histogramme. 

Die grundlegenden Ideen zur Animation solcher Plots bleiben dieselben (`drawframe`-Funktion etc.), die Details (wie update ich die Plotdaten) schauen Sie aber ggf. am besten im WWW nach. Hier einige Links:

- [Animation von Scatter-Plots](https://stackoverflow.com/questions/9401658/how-to-animate-a-scatter-plot)
- [Animation von Histogrammen](https://matplotlib.org/stable/gallery/animation/animated_histogram.html)
- [Animation von Balkendiagrammen](https://stackoverflow.com/questions/42056347/how-to-animate-a-bar-plot)
- [Animation in 3d](https://matplotlib.org/stable/gallery/animation/random_walk.html)

## A.2 Videoformate zum Abspeichern von Animationen

In welchen Videoformaten Animationen abgespeichert werden können (`animation.save(...)`-Kommando), hängt von der Installation von teilweise externen Programmen auf Ihrem System ab. Am gebräuchlichsten ist das Abspeichern als bewegte `gif`-Dateien oder als `mp4`-Videodateien.

Das `gif`-Format sollte auf allen Systemen durch die `Python Image Library` verfügbar sein. Für `mp4` werden externe Programme benötigt.

Die folgende Zelle testet, welche Formate auf Ihren System verfügbar sind. Für weitere Informationen zu dem Thema siehe [die offizielle Dokumentation zu den Animation Writers](https://matplotlib.org/stable/users/explain/animations/animations.html#animation-writers).

In [ ]:
# Einfacher Test, ob auf Ihrem System das gif und/oder das mp4-Format zum Abspeichern
# von Animationen verfügbar sind.

import matplotlib.animation as ma

# Teste of PIL zum Abspeichern von bewegten gifs:
if ma.writers.is_available('pillow'):
    print('Bewegte GIFs können mit der PIL abgespeichert werden.')
    print("Ein Beispielbefehl: anim.save('test.gif', writer='pillow', fps=fps)")
else:
    print('PIL ist bei Ihnen nicht verfügbar.')

if ma.writers.is_available('ffmpeg'):
    print('MPEG4-Videodateien und viele andere Formate können mit ffmpeg abgespeichert werden.')
    print("Ein Beispielbefehl: anim.save('test.mp4', writer=ma.FFMpegWriter(fps=20))")
else:
    print('FFMpeg ist bei Ihnen nicht verfügbar.')

if ma.writers.is_available('imagemagick'):
    print('MPEG4-Videodateien und viele andere Formate können mit ImageMagick abgespeichert werden.')
    print("Ein Beispielbefehl: anim.save('test.mp4', writer=ma.ImageMagickWriter(fps=20))")
else:
    print('ImageMagick ist bei Ihnen nicht verfügbar.')